In [1]:

import datetime as dt
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import h2o
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Polygon
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from statsmodels.tools.tools import add_constant

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from h2o.automl import H2OAutoML
# from tpot import TPOTClassifier
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,ParameterGrid
import xgboost
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
import sys
from sklearn.mixture import GaussianMixture
from sklearn import metrics
import os
import mlflow


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")
os.environ["LOGNAME"] = "ML_Project"
# mlflow.create_experiment(name="ML-Project-Group-6")  #create if you haven't create
mlflow.set_experiment(experiment_name="ML-Project-Group-6")

<Experiment: artifact_location='file:///C:/Users/PREDATOR/Music/ML_project/mlflowruns/273055796817312252', creation_time=1701179561364, experiment_id='273055796817312252', last_update_time=1701179561364, lifecycle_stage='active', name='ML-Project-Group-6', tags={}>

In [3]:
accidents_sub = pd.read_csv("sub_accidents.csv")

In [4]:
accidents_sub = accidents_sub[accidents_sub["Start_Year"]>2020]

In [5]:
accidents_sub.shape

(3047692, 38)

In [6]:
X = accidents_sub.drop(columns=['ID', 'Severity', 'Delay(min)', 'Street', 'City', 'County', 'State', 'Zipcode',
       'Start_Year', 'Start_Month', 'Start_Day', 'Start_Lat', 'Start_Lng',
       'Start_time','Distance(km)', 'Delay_ln', 'Severity_new'])
Y = accidents_sub.loc[:, ['Severity', 'Severity_new', 'Delay(min)', 'Delay_ln', 'Distance(km)']]


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y["Severity_new"], test_size = 0.10, random_state=370, stratify=Y["Severity_new"])

assert len(X_train)  == len(Y_train)
assert len(X_test)   == len(Y_test)
X.shape, Y.shape,X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((3047692, 21),
 (3047692, 5),
 (2742922, 21),
 (304770, 21),
 (2742922,),
 (304770,))

In [8]:
'''# fixing imbalance by under sampling (Tomek links) (it didn't help)
tl = TomekLinks(n_jobs=-1, sampling_strategy="Majority")

# fit predictor and target variable
X_train_tl, Y_train_tl = tl.fit_resample(X_train, Y_train["Severity_new"])'''

'# fixing imbalance by under sampling (Tomek links) (it didn\'t help)\ntl = TomekLinks(n_jobs=-1, sampling_strategy="Majority")\n\n# fit predictor and target variable\nX_train_tl, Y_train_tl = tl.fit_resample(X_train, Y_train["Severity_new"])'

In [9]:
# fixing imbalance by under sampling (random under sampling)
rus = RandomUnderSampler(random_state=370, replacement=False)

# fit predictor and target varialbe
X_train_rus, Y_train_rus = rus.fit_resample(X_train, Y_train)

X.shape, Y.shape, X_train_rus.shape, Y_train_rus.shape

((3047692, 21), (3047692, 5), (490126, 21), (490126,))

I tried to fit a regression model with Delay and distance as labels but no appropriate model have found.

In [10]:
'''model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=370)

model.fit(X_train, Y_train["Distance(km)"])

# Make predictions on the validation set
y_train_pred = model.predict(X_train)

# Calculate metrics
mse = mean_squared_error(Y_train["Distance(km)"], y_train_pred)
r2 = r2_score(Y_train["Distance(km)"], y_train_pred)
print(f'Mean Squared Error on Test Set: {mse}')
print(f'R_squared on Test Set: {r2}')'''




'model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=370)\n\nmodel.fit(X_train, Y_train["Distance(km)"])\n\n# Make predictions on the validation set\ny_train_pred = model.predict(X_train)\n\n# Calculate metrics\nmse = mean_squared_error(Y_train["Distance(km)"], y_train_pred)\nr2 = r2_score(Y_train["Distance(km)"], y_train_pred)\nprint(f\'Mean Squared Error on Test Set: {mse}\')\nprint(f\'R_squared on Test Set: {r2}\')'

In [11]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print("Device: ", device)

Device:  cpu


In [12]:
Y_train.value_counts()

0    2497859
1     245063
Name: Severity_new, dtype: int64

In [13]:
from mlflow.data.pandas_dataset import PandasDataset


lr = LogisticRegression(random_state=370, n_jobs=-1)
rf = RandomForestClassifier(random_state=370, n_jobs=-1, oob_score=True)
sgb = xgboost.XGBClassifier(max_depth=1, n_estimators=200, n_jobs=-1)
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200, learning_rate=0.001, random_state=370)

# models = [ lr, rf, sgb, ada]
models = {"Logistic Regression": lr, "Random Forest":rf, "XGBoost":sgb, "Adaboost":ada}

dataset: PandasDataset = mlflow.data.from_pandas(X_train_rus, source='')


#perform cross validation using KFold
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits = 5, shuffle = True, random_state=370)
for name,model in models.items():
    with mlflow.start_run(run_name=f"Model {name}", nested=True):
        mlflow.log_input(dataset, context="training")
        score = cross_val_score(model, X_train_rus, Y_train_rus, cv=kfold, scoring='accuracy', n_jobs=-1)  #f1, recall, precision, accuracy
        np.set_printoptions(precision = 2)
        print("Model: ", model, "Scores: ", score, "- Scores mean: ", score.mean(), "- Scores std (lower better):", score.std()) 
        params = {"Model":model, "Scores":score, "Scores mean":score.mean(), "Scores std-lower better ":score.std()}
        mlflow.log_params(params=params)
        fit = model.fit(X_train_rus, Y_train_rus)
        yhat = fit.predict(X_test)
        conf_matrix = confusion_matrix(Y_test, yhat)
        true_positive = conf_matrix[0][0]
        true_negative = conf_matrix[1][1]
        false_positive = conf_matrix[0][1]
        false_negative = conf_matrix[1][0]
        mlflow.log_metric("true_positive", true_positive)
        mlflow.log_metric("true_negative", true_negative)
        mlflow.log_metric("false_positive", false_positive)
        mlflow.log_metric("false_negative", false_negative)
        print(confusion_matrix(Y_test, yhat).ravel())
        mlflow.log_metric("ROCAUC_score", roc_auc_score(Y_test, yhat))
        print('ROCAUC score:',roc_auc_score(Y_test, yhat))      
        print(classification_report(Y_test, yhat))
        cr = classification_report(Y_test, yhat, output_dict=True) 
        mlflow.log_metric("accuracy", cr.pop("accuracy"))
        for class_or_avg, metrics_dict in cr.items():
            for metric, value in metrics_dict.items():
                mlflow.log_metric(class_or_avg + '_' + metric,value)     
        print(classification_report(Y_test, yhat))
        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
        mlflow.sklearn.log_model(fit, name)

c:\Users\PREDATOR\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\PREDATOR\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.o

Model:  LogisticRegression(n_jobs=-1, random_state=370) Scores:  [0.82 0.82 0.82 0.82 0.82] - Scores mean:  0.8173979713900781 - Scores std (lower better): 0.0014134375994906685
[234457  43084   5753  21476]
ROCAUC score: 0.8167415944868098
              precision    recall  f1-score   support

           0       0.98      0.84      0.91    277541
           1       0.33      0.79      0.47     27229

    accuracy                           0.84    304770
   macro avg       0.65      0.82      0.69    304770
weighted avg       0.92      0.84      0.87    304770

              precision    recall  f1-score   support

           0       0.98      0.84      0.91    277541
           1       0.33      0.79      0.47     27229

    accuracy                           0.84    304770
   macro avg       0.65      0.82      0.69    304770
weighted avg       0.92      0.84      0.87    304770

Model saved in run 820d1f0eec564aa39b7b9875d2a25628


c:\Users\PREDATOR\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Model:  RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=370) Scores:  [0.84 0.84 0.84 0.84 0.84] - Scores mean:  0.840742989697475 - Scores std (lower better): 0.0018569554764470812
[239351  38190   4804  22425]
ROCAUC score: 0.8429845833134345
              precision    recall  f1-score   support

           0       0.98      0.86      0.92    277541
           1       0.37      0.82      0.51     27229

    accuracy                           0.86    304770
   macro avg       0.68      0.84      0.71    304770
weighted avg       0.93      0.86      0.88    304770

              precision    recall  f1-score   support

           0       0.98      0.86      0.92    277541
           1       0.37      0.82      0.51     27229

    accuracy                           0.86    304770
   macro avg       0.68      0.84      0.71    304770
weighted avg       0.93      0.86      0.88    304770

Model saved in run ad890249007f4312a99b20f8a0aa713a
Model:  XGBClassifier(base_score=N

In [15]:
with mlflow.start_run(run_name=f"Grid-CV-RF", nested=True):
    mlflow.log_input(dataset, context="training")
    # mlflow.sklearn.autolog()
    param_grid = {"n_estimators": [100, 200, 300], "criterion": ["gini", "entropy"], "max_depth": np.arange(1, 10)}
    mlflow.log_params(params=param_grid)
    model = RandomForestClassifier(n_jobs=-1, random_state=370, oob_score=True)
    grid = GridSearchCV(model, param_grid, refit=True, n_jobs=-1)
    grid.fit(X_train_rus, Y_train_rus)
    print(grid.best_params_)
    yhat = grid.predict(X_test)
    grid_best_model = grid.best_estimator_
    print(grid_best_model)
    mlflow.log_params(params= grid.best_params_)
    mlflow.log_metric('best score', grid.best_score_)
    # mlflow.sklearn.log_model(grid_best_model, "gridcv_classifier_model")
    print(classification_report(Y_test, yhat))
    # print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
    print(classification_report(Y_test, yhat))

In [16]:
with mlflow.start_run(run_name=f"Random-Forest-Normal-RS", nested=True):
    mlflow.log_input(dataset, context="training")
    model = RandomForestClassifier(n_estimators= 200, criterion="gini", max_depth= 9, n_jobs=-1, random_state=370)
    params = {"n_estimators":200, "criterion":"gini", "max_depth":9, "n_jobs":-1, "random_state":370}
    mlflow.log_params(params=params)
    model.fit(X_train_rus, Y_train_rus)
    yhat = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
    mlflow.log_metric("true_positive", tp)
    mlflow.log_metric("true_negative", tn)
    mlflow.log_metric("false_positive", fp)
    mlflow.log_metric("false_negative", fn)
    print("True Negative = ",tn, " False Negative = ",fn, "True Positive = ", tp, "False Positive = ", fn)
    print(classification_report(Y_test, yhat))
    cr = classification_report(Y_test, yhat, output_dict=True)
    mlflow.log_metric("accuracy", cr.pop("accuracy"))
    for class_or_avg, metrics_dict in cr.items():
        for metric, value in metrics_dict.items():
            mlflow.log_metric(class_or_avg + '_' + metric,value)
    mlflow.sklearn.log_model(model, "rf_normal")
    mlflow.sklearn.save_model(model, "rf_normal")

True Negative =  243805  False Negative =  6193 True Positive =  21036 False Positive =  6193
              precision    recall  f1-score   support

           0       0.98      0.88      0.92    277541
           1       0.38      0.77      0.51     27229

    accuracy                           0.87    304770
   macro avg       0.68      0.83      0.72    304770
weighted avg       0.92      0.87      0.89    304770



In [17]:
dataset: PandasDataset = mlflow.data.from_pandas(X_train, source='')
with mlflow.start_run(run_name=f"Random-Forest-Balanced-Subsample", nested=True):
    mlflow.log_input(dataset, context="training")
    model = RandomForestClassifier(n_estimators= 200, criterion="gini", max_depth= 9, n_jobs=-1, random_state=370, class_weight="balanced_subsample")
    params = {"n_estimators":200, "criterion":"gini", "max_depth":9, "n_jobs":-1, "random_state":370, "class_weight":"balanced_subsample"}
    mlflow.log_params(params=params)
    model.fit(X_train, Y_train)
    yhat = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
    mlflow.log_metric("true_positive", tp)
    mlflow.log_metric("true_negative", tn)
    mlflow.log_metric("false_positive", fp)
    mlflow.log_metric("false_negative", fn)
    print("True Negative = ",tn, " False Negative = ",fn, "True Positive = ", tp, "False Positive = ", fn)
    print(classification_report(Y_test, yhat))
    cr = classification_report(Y_test, yhat, output_dict=True)
    mlflow.log_metric("accuracy", cr.pop("accuracy"))
    for class_or_avg, metrics_dict in cr.items():
        for metric, value in metrics_dict.items():
            mlflow.log_metric(class_or_avg + '_' + metric,value)
    mlflow.sklearn.log_model(model, "rf_balanced_sub")
    mlflow.sklearn.save_model(model, "rf_balanced_sub")

c:\Users\PREDATOR\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\PREDATOR\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.o

True Negative =  243897  False Negative =  6206 True Positive =  21023 False Positive =  6206
              precision    recall  f1-score   support

           0       0.98      0.88      0.92    277541
           1       0.38      0.77      0.51     27229

    accuracy                           0.87    304770
   macro avg       0.68      0.83      0.72    304770
weighted avg       0.92      0.87      0.89    304770



In [18]:
with mlflow.start_run(run_name=f"Random-Forest-Balanced", nested=True):
    mlflow.log_input(dataset, context="training")
    model = RandomForestClassifier(n_estimators= 200, criterion="gini", max_depth= 9, n_jobs=-1, random_state=370, class_weight="balanced")
    params = {"n_estimators":200, "criterion":"gini", "max_depth":9, "n_jobs":-1, "random_state":370, "class_weight":"balanced"}
    mlflow.log_params(params=params)
    model.fit(X_train, Y_train)
    yhat = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
    mlflow.log_metric("true_positive", tp)
    mlflow.log_metric("true_negative", tn)
    mlflow.log_metric("false_positive", fp)
    mlflow.log_metric("false_negative", fn)
    print("True Negative = ",tn, " False Negative = ",fn, "True Positive = ", tp, "False Positive = ", fn)
    print(classification_report(Y_test, yhat))
    cr = classification_report(Y_test, yhat, output_dict=True)
    mlflow.log_metric("accuracy", cr.pop("accuracy"))
    for class_or_avg, metrics_dict in cr.items():
        for metric, value in metrics_dict.items():
            mlflow.log_metric(class_or_avg + '_' + metric,value)
    mlflow.sklearn.log_model(model, "rf_balanced")
    mlflow.sklearn.save_model(model, "rf_balanced")

True Negative =  243634  False Negative =  6182 True Positive =  21047 False Positive =  6182
              precision    recall  f1-score   support

           0       0.98      0.88      0.92    277541
           1       0.38      0.77      0.51     27229

    accuracy                           0.87    304770
   macro avg       0.68      0.83      0.72    304770
weighted avg       0.92      0.87      0.89    304770



In [30]:
# Range of clusters to test
n_clusters_range = range(1, 11)
bic_scores = []

# Fit the model for each number of clusters and compute BIC
with mlflow.start_run(run_name=f"GMM-Clustering", nested=True):
    mlflow.log_params(params={"n_cluster":n_clusters_range})
    for n_clusters in n_clusters_range:
        gmm = GaussianMixture(n_components=n_clusters, random_state=42)
        gmm.fit(X_train)
    
        # BIC (Bayesian Information Criterion) score
        bic_scores.append(gmm.bic(X_train))

    # Plot the elbow plot
    # plt.figure(figsize=(10, 6))
    # plt.plot(n_clusters_range, bic_scores, marker='o')
    # plt.title('Elbow Plot for Gaussian Mixture Model')
    # plt.xlabel('Number of Clusters')
    # plt.ylabel('BIC Score')
    # plt.grid(True)
    # plt.show()

    # Choose the optimal number of clusters based on the elbow plot
    optimal_n_clusters = np.argmin(bic_scores) + 1  # +1 because Python is zero-indexed
    print(f'Optimal Number of Clusters: {optimal_n_clusters}')
    mlflow.log_metrics("optimal_no_of_cluster", optimal_n_clusters)

    # Train the model with the optimal number of clusters
    optimal_gmm = GaussianMixture(n_components=optimal_n_clusters, random_state=42)
    optimal_gmm.fit(X_train)

    # Predict the clusters for the testing data
    test_labels = optimal_gmm.predict(X_test)

    # Evaluate the model on the testing data
    test_score = optimal_gmm.score(X_test)
    silhouette_score = metrics.silhouette_score(X_test, test_labels)

    print(f'Log-Likelihood on Test Data: {test_score}')
    print(f'Silhouette Score on Test Data: {silhouette_score}')
    mlflow.log_metrics('test_scores', test_score)
    mlflow.log_metrics('silhouette_score', silhouette_score)
    mlflow.sklearn.log_model(optimal_gmm, "gaussian_mixture")
